In [1]:
import pandas as pd
import os
import numpy as np
import geopandas as gpd
import folium

## Organizing coordinates

In [2]:
os.chdir('data')

In [3]:
os.listdir()
coords_fn = 'Koordinates+Elevation.xlsx'
coords_df = pd.read_excel(coords_fn)
coords_df = coords_df.iloc[:,:-1]

col_names_list = ['WellID', 'CaseHeight', 'E', 'N', 'Type']
names_dict = {}
for i in range(len(col_names_list)):
    names_dict [coords_df.iloc[:,i].name] = col_names_list[i]

coords_df = coords_df.rename(columns = names_dict)
coords_df.head()

,WellID,CaseHeight,E,N,Type
0,G1,119.49,5424455.69,5648391.10,NaN
1,G2,118.83,5424474.00,5648390.00,no existing well
2,G3,118.73,5424491.00,5648392.00,no existing well
3,G4,119.19,5424509.75,5648392.55,NaN
4,G4 neu,120.09,NaN,NaN,NaN


## Coordinates completion

After organizing the table above mentioned, we went to the field and collected coordinates manually. Some of them were computed directly in the field by measuring the distance from the well with missing coordinates to a well with known coordinates. Some others had no well closeby, therefore the group measured the distance to the closest corner of the site or of the building and saved it to compute the coordinates later on. An *e-mail was sent to Zhao* asking for a georrefenced file. The group intends to use this file to extract the coordinates of the known points and compute the missing well coordinates.

Thomas said by e-mail on the 28-10-2022 that the coordinate system of the area is Gauss-Krüger.

Now below, I compute the partial coordinates and convert it from Gauss-Krüger UTM tto WGS 1984.

According to [epsg](https://www.geoportal.rlp.de/mediawiki/index.php/EPSG-Codes/de), the epsg of the coordinate system is 31469. We convert from UTM to GCS WGS 1984, epsg 4326. It can also be manually converted from [here](https://www.koordinaten-umrechner.de/decimal/51.000000,10.000000?karte=OpenStreetMap&zoom=8).


In [2]:
os.chdir('D:/Repos/PirnaCaseStudy/Data/Databases')
os.listdir()

['DiverID.csv', 'WellID.csv']

Read data

In [18]:
#define input epsg
epsg = 31469
#define output epsg
epsg_ = 4326 #WGS 1984

WellID_df = pd.read_csv('WellID.csv')

WellID_df = WellID_df[(~WellID_df.E.isna() ) &
                      (~WellID_df.CaseHeight.isna()
)].reset_index(drop=1)


#generating points
WellID_df['geometry'] = gpd.points_from_xy(WellID_df.E, WellID_df.N, crs = 'EPSG:{}'.format(epsg) )
WellID_df = WellID_df.drop(columns = ['E', 'N'])

WellID_gdf = gpd.GeoDataFrame(WellID_df, crs = 'EPSG:{}'.format(epsg)) #generate gdf
WellID_gdf = WellID_gdf.to_crs(epsg_) #convert to output epsg

WellID_gdf.head()

,WellID,CaseHeight,Type,Depth,FilterTop,FilterBase,Lithology,geometry
0,G1,119.49,Well,NaN,NaN,NaN,NaN,POINT (-29.19015 0.00407)
1,G2,118.83,borehole,NaN,NaN,NaN,NaN,POINT (-29.19015 0.00407)
2,G3,118.73,borehole,NaN,NaN,NaN,NaN,POINT (-29.19015 0.00407)
3,G4,119.19,Well,NaN,NaN,NaN,NaN,POINT (-29.19015 0.00407)
4,G4neu,10.09,NaN,NaN,NaN,NaN,NaN,POINT (-29.19015 0.00407)


In [19]:
WellID_df

,WellID,CaseHeight,Type,Depth,FilterTop,FilterBase,Lithology,geometry
0,G1,119.490,Well,NaN,NaN,NaN,NaN,POINT (13.923 50.966)
1,G2,118.830,borehole,NaN,NaN,NaN,NaN,POINT (13.923 50.966)
2,G3,118.730,borehole,NaN,NaN,NaN,NaN,POINT (13.923 50.966)
3,G4,119.190,Well,NaN,NaN,NaN,NaN,POINT (13.923 50.966)
4,G4neu,10.090,NaN,NaN,NaN,NaN,NaN,POINT (13.923 50.966)
5,G5,118.933,borehole,NaN,NaN,NaN,NaN,POINT (13.923 50.966)
6,G10,119.290,Well,NaN,NaN,NaN,NaN,POINT (13.923 50.966)
7,G11,116.950,Well,NaN,NaN,NaN,NaN,POINT (13.924 50.966)
8,G12,116.570,Well,NaN,NaN,NaN,NaN,POINT (13.923 50.966)
9,G13,116.970,Well,NaN,NaN,NaN,NaN,POINT (13.923 50.966)


Transform it to geodataframe convert coordinates, and plot it.

# Plotting

In [4]:
# Create a geometry list from the GeoDataFrame
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in WellID_gdf.geometry]

map_center = WellID_gdf.geometry.y.mean(), WellID_gdf.geometry.x.mean() #in folium y comes before x

#create object map
map = folium.Map(location=map_center, tiles="OpenStreetMap", zoom_start=19)

i = 0
for coordinates in geo_df_list:
    # Place the markers with the popup labels and data
    map.add_child(
        folium.Marker(
            location = coordinates,
            popup = "Well: " + WellID_gdf.WellID[i]
        )
    )
    i += 1

map

## Saving

In [21]:
os.chdir('../')

In [22]:
os.getcwd()

'D:\\Repos\\PirnaCaseStudy\\Data'

In [15]:
WellID_df['E'], WellID_df['N'] = WellID_gdf.geometry.x, WellID_gdf.geometry.y
WellID_df = WellID_gdf.drop(columns = 'geometry')
WellID_df.head()

,WellID,CaseHeight,Type,Depth,FilterTop,FilterBase,Lithology,E,N
0,G1,119.49,Well,NaN,NaN,NaN,NaN,13.922657,50.965827
1,G2,118.83,borehole,NaN,NaN,NaN,NaN,13.922918,50.965820
2,G3,118.73,borehole,NaN,NaN,NaN,NaN,13.923160,50.965840
3,G4,119.19,Well,NaN,NaN,NaN,NaN,13.923426,50.965848
4,G4neu,10.09,NaN,NaN,NaN,NaN,NaN,13.923416,50.965847


In [24]:
WellID_df.to_csv('WellID.csv', index=False)

## Manual editting

After gathering with Nico on the 09/11/2022, some details were (case hight, depth and filter) were added manually. An e-mail was sent by Nico on the same day asking if Thomas could check it.